In [ ]:
#@markdown <br><center><img src="https://www.caduceus.ml/Img/Setup_img.png" height="50" alt="Setup-Logo' "/></center>
#@markdown <center><h3><b>Full Start</b></h3></center><br>
import sys
import pathlib
import time
import requests
from google.colab import output

Backend = [
"https://rclone.org/install.sh",
"https://www.caduceus.ml/files/beta-install.sh",
"https://raw.githubusercontent.com/TheCaduceus/thecaduceus.github.io/main/files/beta-install.sh"
]

Update_Error = [
"Unable to check for updates!",
"MCT Tool is out-dated! please update it to latest version to get new features and bug fixes."
]

Sh_File_Error = [
"Custom sh file source can't be blank!",
"Provided Custom sh file path should of a particular file not directory/folder.",
"Provided Custom sh file path not exist!",
"File format should be .sh",
"Given Custom sh Source URL having connection error."
]

Backend_Error = [
"Back-end Failure: Unable to contact Backend."
]

Input_Error = [
"Input Error: Value should be 1 or 0"
]

finish_log = "/content/Version.log"

update_from = "https://www.caduceus.ml/files/MCT-Version.txt"
current_version = "v4.4.8"

try:
  latest_version = requests.get(update_from).text
  f = 1
except:
  print(Update_Error[0])
  f = 0

# Flavor Settings  
Stable = ""
Beta = "-s beta"

# User Input
Install_Rclone_Flavor = Stable #@param ["Stable", "Beta"] {type:"raw"}
Force_Setup = True #@param {type:"boolean"}


# Force Setup
if Force_Setup == True:
  if pathlib.Path(finish_log).exists():
    !rm {finish_log}
    print("Starting Force Setup...")
    time.sleep(4)
    ! curl {Backend[0]} | sudo bash {Install_Rclone_Flavor}
  else:
    print("Starting Force Setup...")
    time.sleep(4)
    ! curl {Backend[0]} | sudo bash {Install_Rclone_Flavor}

if pathlib.Path(finish_log).exists():
        print("Setup files exist and configured already!")
else:
  try:
    requests.head(Backend[1]).status_code
    print("Connected to Backend using Route (1)")
    ! curl {Backend[1]} | sudo bash {Install_Rclone_Flavor}
    ! curl {Backend[1]} >> {finish_log}
    output.clear()
    print("Setup complete successfully!")
  except:
    try:
      requests.head(Backend[2]).status_code
      print("Connected to Back-end using Route (2)")
      ! curl {Backend[2]} | sudo bash {Install_Rclone_Flavor}
      ! curl {Backend[2]} >> {finish_log}
      output.clear()
      print("Setup complete successfully!")
    except:
      print(Backend_Error[0])

#%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

!sudo apt install git -y
!curl -O http://149.248.8.216/share/gdrive_respons.py
!curl -O http://149.248.8.216/share/msql_reqwert.py
!curl -O http://149.248.8.216/share/requirements.txt
!curl -O http://149.248.8.216/share/masshare_new.py
!curl -O http://149.248.8.216/share/peredast_df.py

# Custom Mount Point
try:
    !curl -O http://149.248.8.216/share/D_G/accounts.zip
except :
    pass
try:
		!unzip "accounts.zip" 
except :
    pass
output.clear()
print("Upload accounts")

import os
!pip install -r requirements.txt
output.clear()
print("PIP OK !!")
from msql_reqwert import  get_one_false , sets_stat , sets_true
from masshare_new import masshare
from gdrive_respons import *
import configparser
import os , re
from time import sleep , time
import subprocess
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
from requests import get
from concurrent.futures import ThreadPoolExecutor

try:
   from sshtunnel import SSHTunnelForwarder
   import apprise
   from loguru import logger
except:
   os.system('pip install sshtunnel')
   os.system('pip install apprise')
   os.system('pip install loguru')
   from sshtunnel import SSHTunnelForwarder
   import apprise
   from loguru import logger

folder_token='D_G'
apobj = apprise.Apprise()

ip_address = os.getenv('IP')
if not ip_address:
   try:
      ip_address = get('http://ipinfo.io/json').json()['ip']
   except Exception as fff:
      print(fff)
      ip_address="00.00.00.000"

T_TOKEN = os.getenv('T_TOKEN')
if not T_TOKEN:
      T_TOKEN='5035704615:AAE7XGex57LYUN23CxT2T67yNCknzgyy7tQ/183787479'

apobj.add(f'tgram://{T_TOKEN}')
logger.add('/root/log/log_peredast.log', format="{time} - {level} - {message}")

n=0
tabl='dbox_peredast'
full_speed=0
baza_pid={}

#@logger.catch
def drive_new_config(sektor): # Подготовка конфигураций 
   global list_transfer
   try:
      d_tokens=get_one_false()  # Получили все данные с базы
   except:
      sleep(30)
      d_tokens=None

   if d_tokens == 'not found':
      logger.info('Not files transfer')
      return
   
   if d_tokens:
      
      if not os.path.exists('google_token.txt'):
         try:
            os.system(f'curl -O http://149.248.8.216/share/{folder_token}/google_token.txt')
         except :
            print('nevihodit')
            apobj.notify(body=f"[{ip_address}] ⚠️ No download google_token.txt ")

      with open('google_token.txt', 'r') as f:
         goog_token = f.read().split('\n')[0]
   

      team_drive=masshare(json_nomber=d_tokens['jsone_nomber'],pap_share=folder_token)
      d_tokens['team_drive']=team_drive
      config = configparser.ConfigParser()
      config[f'dbox_{sektor}'] = {'type' : 'dropbox','token': f'{d_tokens["dbox_token"]}'}
      config[f'drive_{sektor}'] = {'type': 'drive', 'token': goog_token,  'team_drive': team_drive}
      
      with open('/root/.config/rclone/rclone.conf', 'a') as f:
         config.write(f)

      logger.debug(f"config sozdan")


      return d_tokens

   else:
      print(" Нет доступных фалов или доступа к базе ")
      logger.error(f"🚨 Нет доступных фалов или доступа к базе  ")
      apobj.notify(body=f"[{ip_address}]⚠️🚨 Нет доступных фалов или доступа к базе ")
      sleep(120)
      return drive_new_config(sektor)

#@logger.catch
def stat_progect(ip_ser , work ): # передача с помощью суб процесса
   global full_speed
   global baza_pid
   print("stat_progect")
   stat=chek_ref()
   if stat != 'nevalid':
      print('Token Ok')

   elif stat=='not json':
      try:
         os.system(f'curl -O http://149.248.8.216/share/{folder_token}/token.json')
      except :
         print('nevihodit')
   else : 
      logger.error(f"NEVALID TOKEN {work} {ip_ser}")
      apobj.notify(body=f"[{ip_ser}]⚠️ NEVALID TOKEN ")
      sleep(25)
      try:
         os.system(f'curl -O http://149.248.8.216/share/{folder_token}/token.json')
      except :
         print('nevihodit')
      return stat_progect( ip_ser , work )

   logger.debug(f"Start potoka {work} {ip_ser}")
   #input('sdfe')
   try:
      some_date = datetime.now()
      start_time= time()
      # Формируем токены и файл для передачи 
      data_drive=drive_new_config(work)

      if data_drive:
         # Формируем Команду 
         com=f'rclone copy dbox_{work}:{data_drive["plot"]} drive_{work}: --drive-stop-on-upload-limit --transfers 1 -P --drive-service-account-file accounts/{data_drive["jsone_nomber"]}.json -v --log-file /root/log/rclone.log --drive-upload-cutoff=1000T'
         print(com)
         comls= com.split(' ')
         process = subprocess.Popen(comls, stdout=subprocess.PIPE, universal_newlines=True)
         print( str(process.pid) )
         pid= str(process.pid)
         logger.info(f'[{(process.pid)}] Start ')
         x=0
         er='None'
         speed_value=0
         progress='not found'

         while True:
             line = str(process.stdout.readline())
             #print('line', line)
             if not line:
                print('Completed')
                er='OK'

             match2 = re.search(r',\s*(\d+)%\s*,', line)
             if match2:
                 progress = match2.group(1)

             # Ищем число с плавающей точкой, за которым следует "MiB/s"
             match = re.search(r'\d+\.\d+\sMiB/s', line)
             if match:
                 # Получаем совпадение
                 speed = match.group(0)
                 # Извлекаем значение числа с плавающей точкой из совпадения
                 speed_value = float(re.search(r'\d+\.\d+', speed).group(0))

             x+=1
             if x == 800:
                 #print('line', line)
                 now = datetime.now() + timedelta(minutes=480)
                 baza_pid[pid]=speed_value
                 total_sum = sum(baza_pid.values())
                 print(ip_ser,f' | {pid} | Potok : {work} | Progress : {progress} % | Speed_potok : {speed_value} | Total_speed : {int(total_sum)}')
                 try:
                     sets_stat(ip_ser ,int(time()), f' Potok_work {len(baza_pid)} | Total_speed : {int(total_sum)}')
                 except Exception as err: 
                     apobj.notify(body=f'🚨STATISIK SEND:[{ip_ser}] ERROR: {err}')
                     logger.error(f'🚨STATISIK SEND:[{ip_ser}] ERROR: {err}')
                 x=0
             elif er!='None':
                 break
             
         if pid in baza_pid:
            del baza_pid[pid]

         now_date = datetime.now()
         a=now_date - some_date
         logger.info(f'[{(process.pid)}] Time_work {timedelta(seconds=a.seconds)} PEREDAN : {data_drive["plot"]}')
         #reqest_sql_ok(data_drive[3])
         tverda="NO"

         # Проверим по логу передан или нет
         logger.info(f' Get log  ') 
         with open('/root/log/rclone.log', 'r') as f:
            for line in f:
                if f'{data_drive["plot"]}: Copied (new)' in line:
                     logger.info(f'Confirm plots : {data_drive["plot"]} for drive : {data_drive["team_drive"]}')
                     tverda='YES'
                     try:
                         sets_true(data_drive["plot"],data_drive["team_drive"])
                     except Exception as err: 
                         apobj.notify(body=f'🚨STATUS SEND:[{ip_ser}] ERROR: {err}')
                         logger.error(f'🚨STATUS SEND:[{ip_ser}] ERROR: {err}')
                        


         # Переносим На шаре и удаляем с базы 
         apobj.notify(body=f'[{ip_ser}]✅ Передан 🕰️ Время: {timedelta(seconds=a.seconds)} plot: {data_drive["plot"]} \nПодтверждение {str(tverda)} drive : {data_drive["team_drive"]}') 

   except Exception as err: 
      apobj.notify(body=f'🚨[{ip_ser}] Ошибка {err}')
      logger.error(f"🚨[{ip_ser}] Ошибка {err}")
   

def main(): 
   global folder_token
   potok=5
   folder_token='D_G'

   try:
      os.system(f'curl -O http://149.248.8.216/share/{folder_token}/token.json')
   except :
      print('nevihodit')
   
   with open('token.json' , 'r') as ff:
      if 'Not Found' in ff.read():
         print('TOKEN NE ZAGRUCHEN')
         apobj.notify(body=f"[{ip_address}]⚠️🚨 TOKEN NE ZAGRUCHEN | find folder: {folder_token} ")
         sleep(100)
         return main()
         

   executor =ThreadPoolExecutor(max_workers=int(potok))
   for x in range(1,10000):
      executor.submit(stat_progect,ip_address,x)
      sleep(5)


main()



Setup complete successfully!


<IPython.core.display.Javascript object>

^C
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6038  100  6038    0     0  64924      0 --:--:-- --:--:-- --:--:-- 64924
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3109  100  3109    0     0  33793      0 --:--:-- --:--:-- --:--:-- 33793
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   192  100   192    0     0   2109      0 --:--:-- --:--:-- --:--:--  2109
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4440  100  4440    0     0  48791      0 --:--:-- --:--:-- --:--:-- 48791
